# Tiến hành lấy dữ liệu từ trang kaggle

In [ ]:
#Making a .kaggle folder
# !mkdir ~/.kaggle

In [ ]:
#Pasting Copying the kaggle.json file to .kaggle folder
# !cp kaggle.json ~/.kaggle/

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#Giving access to read/wirte the kaggle.json file
# !chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# #Updating the kaggle to the latest version
# !pip uninstall -y kaggle
# !pip install --upgrade pip
# !pip install kaggle==1.5.12
# !kaggle -v

In [ ]:
#Dowloading the competition dataset
# !kaggle competitions download -c plant-pathology-2020-fgvc7

In [ ]:
#Unzipping the download dataset into google drive
# !unzip /content/plant-pathology-2020-fgvc7.zip -d "/content/drive/MyDrive/PPNCKH/DataProject"

# Các thư viện sử dụng

In [ ]:
# Importing necessary libraries
import tensorflow as tf # For Deep Learning
from tensorflow import keras # For reading dataset, creating & training models

import numpy as np # for some matrix calculations and other small stuffs
import pandas as pd # reading csv dataset

import matplotlib.pyplot as plt # for creating and plotting graphs
import plotly.express as px # for creating interactive graphs
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split # splitting our dataset

# Small utilities
import datetime
import os

import cv2

from keras.preprocessing.image import ImageDataGenerator,array_to_img, img_to_array, load_img 

../input/plant-pathology-2020-fgvc7/test.csv

# Tiến hành quá trình phân tích dữ liệu

In [ ]:
# Reading the trainning data
dataset = pd.read_csv("../input/plant-pathology-2020-fgvc7/train.csv")

In [ ]:
# fig = px.histogram(dataset, x = 'healthy', title = 'healthy class', color = 'healthy')
# fig.show()

In [ ]:
# fig = px.histogram(dataset, x = 'rust', title = 'Rust Class', color = 'rust')
# fig.show()

In [ ]:
# fig = px.histogram(dataset, x = 'scab', title = 'Scab Class', color = 'scab')
# fig.show()

In [ ]:
# fig = px.histogram(dataset, x = 'multiple_diseases', title = 'Multiple Diseases Class', color = 'multiple_diseases')
# fig.show()

In [ ]:
# labels = dataset.columns[1:] # Getting the labels name
# print(labels)

# sizes = list(dataset.iloc[:, 1:].sum()) # Getting the total number of all classes/labels (Tổng số lượng các hàng có giá trị 1 trong từng cột)
# print(sizes)

# fig = go.Figure(data=[go.Pie(labels = labels, values = sizes)]) # Circle History of total value 
# fig.show()

# Nhận xét dữ liệu:
- Các loại ảnh gắn nhãn sai(xem trong phần thảo luận)
- Các ảnh với các nhãn khác nhau
- Mất cân bằng
# Nêu phương pháp xử lý

In [ ]:
image_batch = dataset['image_id']
label_batch = dataset[['healthy', 'multiple_diseases', 'rust', 'scab']]

img_size= 224

In [ ]:
train_image=[]
for name in dataset['image_id']:
    path= '../input/plant-pathology-2020-fgvc7/images/'+name+'.jpg'
    img=cv2.imread(path)
    image=cv2.resize(img,(img_size,img_size),interpolation=cv2.INTER_AREA)
    train_image.append(image)

fig, ax = plt.subplots(1, 4, figsize=(15, 15))
for i in range(4):
    ax[i].set_axis_off()
    ax[i].imshow(train_image[i])
    
from keras.preprocessing.image import img_to_array

x_train = np.ndarray(shape=(len(train_image), img_size, img_size, 3),dtype = np.float32)
i=0
for image in train_image:
    x_train[i]=img_to_array(image)
    x_train[i]=train_image[i]
    i=i+1

# 255 la thang mau    
x_train=x_train/255
print('Train Shape: {}'.format(x_train.shape))


In [ ]:
y = dataset.copy()
del y['image_id']
y.head()

y_train = np.array(y.values)
print(y_train.shape,y_train[0])

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=115)

x_train.shape, x_val.shape, y_train.shape, y_val.shape

In [ ]:
from imblearn.over_sampling import SMOTE 

sm = SMOTE(random_state = 115) 
 
x_train, y_train = sm.fit_resample(x_train.reshape((-1, img_size * img_size * 3)), y_train)
x_train = x_train.reshape((-1, img_size, img_size, 3))
x_train.shape, y_train.sum(axis=0)

# Sử dụng wandb để vẽ các đồ thị. Mục đích: có nhiều đồ thị trên cùng 1 biểu đồ để so sánh sự tương quan

In [ ]:
# !pip install --upgrade wandb
!pip install wandb --upgrade

In [ ]:
# !pip install wandb==0.9.7

In [ ]:
import wandb
from wandb.keras import WandbCallback

In [ ]:
!wandb login 14f90f3594d09a5e82dc0f95ae79f0ad11b605e9

# Quá trình thí nghiệm trên các mô hình: Xception; Efficientnet B4; Efficientnet B7

## Xception model

In [ ]:
# def create_model_xception():
#     xception = tf.keras.models.Sequential([
#         tf.keras.applications.xception.Xception(include_top=False, weights='imagenet', input_shape=(img_size, img_size, 3)),
#         tf.keras.layers.GlobalAveragePooling2D(),
#         tf.keras.layers.Dense(4, activation='softmax')
#     ])
    
#     xception.layers[0].trainable = False
#     return xception

# model_xception = create_model_xception()
# model_xception.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(name='AUC')])

## Efficientnet model

In [ ]:
!pip install efficientnet

In [ ]:
import efficientnet.tfkeras as efn

### Efficientnet B4

In [ ]:
# model_efficientnet_b4 = tf.keras.Sequential([
#         efn.EfficientNetB4(
#             input_shape=(img_size, img_size, 3),
#             weights='imagenet',
#             include_top=False
#         ),
#         keras.layers.GlobalAveragePooling2D(),
#         keras.layers.Dense(4, activation='softmax')
#     ])
        

# model_efficientnet_b4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(name='AUC')])

### Efficientnet B7

In [ ]:
# model_efficientnet_b7 = tf.keras.Sequential([
#         efn.EfficientNetB7(
#             input_shape=(img_size, img_size, 3),
#             weights='imagenet',
#             include_top=False
#         ),
#         keras.layers.GlobalAveragePooling2D(),
#         keras.layers.Dense(4, activation='softmax')
#     ])
        
# model_efficientnet_b7.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(name='AUC')])


In [ ]:
from keras.layers import Conv2D, MaxPooling2D

model_efficientnet_b7 = tf.keras.Sequential([
        efn.EfficientNetB7(
            input_shape=(img_size, img_size, 3),
            weights='imagenet',
            include_top=False
        ),
        keras.layers.MaxPooling2D(pool_size=(2,2)),
        keras.layers.Flatten(),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(4, activation='softmax')
    ])
        
model_efficientnet_b7.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(name='AUC')])


In [ ]:
model_efficientnet_b7.summary()

# Tiến hành cài đặt các thông số trước khi chạy

In [ ]:
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
LR_reduce=ReduceLROnPlateau(monitor='val_accuracy',
                            factor=.5,
                            patience=10,
                            min_lr=.000001,
                            verbose=1)

ES_monitor=EarlyStopping(monitor='val_loss',
                          patience=20)

reg = .0005

# Thực hiện 2 lần: 
## Thực hiện các chức năng không làm mất cạnh ảnh
## Chức năng thực hiện cắt bớt một phần ảnh
## Kết quả: Bị trúng vào trường hợp overfitting -> Lý do: Lần đầu tiên chỉ áp dụng xoay ảnh và hình như nó không tạo nên ảnh mới khác hoàn toàn
##                                                        Lần thứ hai áp dụng cắt ảnh thì cũng gặp trường hợp tương tự
# Lần 3: Dùng pp lần 2 nhưng áp dụng thêm thay đổi màu sắc ảnh -> Tốt hơn

In [ ]:
datagen = ImageDataGenerator(
#                             rotation_range=45,
#                             shear_range=.25,
#                               zoom_range=.25,
# #     
#                               width_shift_range=.25,
#                               height_shift_range=.25,
    
# #                               rescale=1/255,
                             
                              brightness_range=[.5,1.5], # Luôn sử dụng
    
                              horizontal_flip=True,
                              vertical_flip=True,
                                 
#                               fill_mode='nearest'
#                              featurewise_center=True,
                             samplewise_center=True, ## Test thử
#                              featurewise_std_normalization=True,
#                              samplewise_std_normalization=True,
#                              zca_whitening=True
                              )
#24 16 32 64 128
batch_size = 16
epochs = 300

In [ ]:
img_path = '../input/plant-pathology-2020-fgvc7/images/Train_0.jpg' 

img = load_img(img_path, target_size=(224, 224))
# Convert ht image to Array of dimension (150,150,3)
x   = img_to_array(img)                           
x   = x.reshape((1,) + x.shape)
# Rescale by 1/255

In [ ]:
# iterator
aug_iter = datagen.flow(x, batch_size=1)

# generate samples and plot
fig, ax = plt.subplots(nrows=1, ncols=5, figsize=(15,15))

# generate batch of images
for i in range(5):

    # convert to unsigned integers
    image = next(aug_iter)[0].astype('uint8')

    # plot image
    ax[i].imshow(image)
    ax[i].axis('off')

## Run with xception model

In [ ]:
# ## Initlisazie wandb project
# wandb.init(project='project ppnckh 280522' ,
#            notes='Round 1',
#            name='Train xception model'
#           )

In [ ]:
# wandb_callback = WandbCallback()

In [ ]:
# history_xception = model_xception.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
#                               epochs=epochs,
#                               steps_per_epoch=x_train.shape[0] // batch_size,
#                               verbose=1,
#                               callbacks=[ES_monitor,LR_reduce,wandb_callback],
#                               validation_data=datagen.flow(x_val, y_val,batch_size=batch_size),
#                               validation_steps=x_val.shape[0]//batch_size
#                               )

## Run with efficientnet B4

In [ ]:
# ## Initlisazie wandb project
# wandb.init(project='project ppnckh 280522' ,
#            notes='Round 1',
#            name='Train efficientnet B4 model'
#           )

In [ ]:
# wandb_callback = WandbCallback()

In [ ]:
# history_efficientnet_b4 = model_efficientnet_b4.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
#                               epochs=epochs,
#                               steps_per_epoch=x_train.shape[0] // batch_size,
#                               verbose=1,
#                               callbacks=[ES_monitor,LR_reduce,wandb_callback],
#                               validation_data=datagen.flow(x_val, y_val,batch_size=batch_size),
#                               validation_steps=x_val.shape[0]//batch_size
#                               )

## Run with efficientnet B7

In [ ]:
## Initlisazie wandb project
wandb.init(project='test 090622 2st change data augmentation' ,
           notes='Round 1',
           name='Train efficientnet B7 model 5st'
          )


In [ ]:
wandb_callback = WandbCallback()

In [ ]:
history_efficientnet_b7 = model_efficientnet_b7.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                              epochs=epochs,
                              steps_per_epoch=x_train.shape[0] // batch_size,
                              verbose=1,
                              callbacks=[ES_monitor,LR_reduce,wandb_callback],
                              validation_data=datagen.flow(x_val, y_val,batch_size=batch_size),
                              validation_steps=x_val.shape[0]//batch_size
                              )

In [ ]:

# from keras.layers import Conv2D, MaxPooling2D

# model_efficientnet_b7_2st = tf.keras.Sequential([
#         efn.EfficientNetB7(
#             input_shape=(img_size, img_size, 3),
#             weights='imagenet',
#             include_top=False
#         ),
#         keras.layers.MaxPooling2D(pool_size=(2,2)),
#         keras.layers.Flatten(),
#         keras.layers.BatchNormalization(),
#         keras.layers.Dropout(0.3),
#         keras.layers.Dense(4, activation='softmax')
#     ])
        
# model_efficientnet_b7_2st.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(name='AUC')])


In [ ]:
# ## Initlisazie wandb project
# wandb.init(project='test 090622 1st' ,
#            notes='Round 1',
#            name='Train efficientnet B7 model 2st'
#           )


In [ ]:
# wandb_callback = WandbCallback()

In [ ]:
# model_efficientnet_b7_2st = model_efficientnet_b7_2st.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
#                               epochs=epochs,
#                               steps_per_epoch=x_train.shape[0] // batch_size,
#                               verbose=1,
#                               callbacks=[ES_monitor,LR_reduce,wandb_callback],
#                               validation_data=datagen.flow(x_val, y_val,batch_size=batch_size),
#                               validation_steps=x_val.shape[0]//batch_size
#                               )

# Tiến hành đánh giá kết quả của các mô hình - mô hình được đánh giá dựa trên AUC

## Xception model

### Xử lý đánh giá kết quả

In [ ]:
# pred = model_xception.predict(x_val)

# pred_around = np.zeros_like(pred)
# pred_around[np.arange(len(pred)), pred.argmax(1)] = 1

In [ ]:
# from sklearn.metrics import roc_auc_score

# roc_sum = 0
# for i in range(4):
#     score = roc_auc_score(y_val[:, i], pred_around[:, i])
#     roc_sum += score
#     print(f'{score:.3f}')

# roc_sum /= 4
# print(f'totally:{roc_sum:.3f}')

In [ ]:
# from sklearn.metrics import auc
# from sklearn.metrics import accuracy_score


# print("AUC: ", auc(y_val, pred_around))
# print("Accuracy: ", accuracy_score(y_val.argmax(axis=1), pred_around.argmax(axis=1)))

In [ ]:
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import plot_confusion_matrix

# conf_mat = confusion_matrix(y_val.argmax(axis=1), pred_around.argmax(axis=1))
# import seaborn as sns
# labels = ['Healthy', 'Multiple','Rust','Scab']
# sns.heatmap(conf_mat, xticklabels=labels, yticklabels=labels, annot=True)

### Đánh giá file kết quả

In [ ]:
# dataset_test = pd.read_csv("../input/plant-pathology-2020-fgvc7/test.csv")

# image_batch_test = dataset_test['image_id']

In [ ]:
# test_image=[]
# for name in dataset_test['image_id']:
#     path='../input/plant-pathology-2020-fgvc7/images/'+name+'.jpg'
#     img=cv2.imread(path)
#     image=cv2.resize(img,(img_size,img_size),interpolation=cv2.INTER_AREA)
#     test_image.append(image)
# fig, ax = plt.subplots(1, 4, figsize=(15, 15))
# for i in range(4):
#     ax[i].set_axis_off()
#     ax[i].imshow(test_image[i])    

# from keras.preprocessing.image import img_to_array

# x_test = np.ndarray(shape=(len(test_image), img_size, img_size, 3),dtype = np.float32)
# i=0
# for image in test_image:
#     x_test[i]=img_to_array(image)
#     x_test[i]=test_image[i]
#     i=i+1
    
# x_test=x_test/255
# print('Test Shape: {}'.format(x_test.shape))

In [ ]:
# result = model_xception.predict(x_test)

# res = pd.DataFrame()
# res['image_id'] = image_batch_test
# res['healthy'] = result[:, 0]
# res['multiple_diseases'] = result[:, 1]
# res['rust'] = result[:, 2]
# res['scab'] = result[:, 3]
# res.to_csv('submission_xception.csv', index=False)
# res.head(10)

## Efficientnet B4 model

### Xử lý đánh giá kết quả

In [ ]:
# pred = model_efficientnet_b4.predict(x_val)

# pred_around = np.zeros_like(pred)
# pred_around[np.arange(len(pred)), pred.argmax(1)] = 1

In [ ]:
# from sklearn.metrics import roc_auc_score

# roc_sum = 0
# for i in range(4):
#     score = roc_auc_score(y_val[:, i], pred_around[:, i])
#     roc_sum += score
#     print(f'{score:.3f}')

# roc_sum /= 4
# print(f'totally:{roc_sum:.3f}')

In [ ]:
# from sklearn.metrics import auc
# from sklearn.metrics import accuracy_score


# print("AUC: ", auc(y_val.argmax(axis=1), pred_around.argmax(axis=1)))
# print("Accuracy: ", accuracy_score(y_val.argmax(axis=1), pred_around.argmax(axis=1)))

In [ ]:
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import plot_confusion_matrix

# conf_mat = confusion_matrix(y_val.argmax(axis=1), pred_around.argmax(axis=1))
# import seaborn as sns
# labels = ['Healthy', 'Multiple','Rust','Scab']
# sns.heatmap(conf_mat, xticklabels=labels, yticklabels=labels, annot=True)

### Đánh giá file kết quả

In [ ]:
# dataset_test = pd.read_csv("../input/plant-pathology-2020-fgvc7/test.csv")

# image_batch_test = dataset_test['image_id']

In [ ]:
# test_image=[]
# for name in dataset_test['image_id']:
#     path='../input/plant-pathology-2020-fgvc7/images/'+name+'.jpg'
#     img=cv2.imread(path)
#     image=cv2.resize(img,(img_size,img_size),interpolation=cv2.INTER_AREA)
#     test_image.append(image)
# fig, ax = plt.subplots(1, 4, figsize=(15, 15))
# for i in range(4):
#     ax[i].set_axis_off()
#     ax[i].imshow(test_image[i])    

# from keras.preprocessing.image import img_to_array

# x_test = np.ndarray(shape=(len(test_image), img_size, img_size, 3),dtype = np.float32)
# i=0
# for image in test_image:
#     x_test[i]=img_to_array(image)
#     x_test[i]=test_image[i]
#     i=i+1
    
# x_test=x_test/255
# print('Test Shape: {}'.format(x_test.shape))

In [ ]:
# result = model_efficientnet_b4.predict(x_test)

# res = pd.DataFrame()
# res['image_id'] = image_batch_test
# res['healthy'] = result[:, 0]
# res['multiple_diseases'] = result[:, 1]
# res['rust'] = result[:, 2]
# res['scab'] = result[:, 3]
# res.to_csv('submission_efficientnet_b4.csv', index=False)
# res.head(10)



## Efficientnet B7 model

### Xử lý đánh giá kết quả

In [ ]:
pred = model_efficientnet_b7.predict(x_val)

pred_around = np.zeros_like(pred)
pred_around[np.arange(len(pred)), pred.argmax(1)] = 1

In [ ]:
# from sklearn.metrics import auc
# from sklearn.metrics import accuracy_score


# print("AUC: ", auc(y_val.argmax(axis=1), pred_around.argmax(axis=1)))
# print("Accuracy: ", accuracy_score(y_val.argmax(axis=1), pred_around.argmax(axis=1)))

In [ ]:
from sklearn.metrics import roc_auc_score

roc_sum = 0
for i in range(4):
    score = roc_auc_score(y_val[:, i], pred_around[:, i])
    roc_sum += score
    print(f'{score:.3f}')

roc_sum /= 4
print(f'totally:{roc_sum:.3f}')

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

conf_mat = confusion_matrix(y_val.argmax(axis=1), pred_around.argmax(axis=1))
import seaborn as sns
labels = ['Healthy', 'Multiple','Rust','Scab']
sns.heatmap(conf_mat, xticklabels=labels, yticklabels=labels, annot=True)

### Đánh giá file kết quả

In [ ]:
dataset_test = pd.read_csv("../input/plant-pathology-2020-fgvc7/test.csv")

image_batch_test = dataset_test['image_id']

In [ ]:
test_image=[]
for name in dataset_test['image_id']:
    path='../input/plant-pathology-2020-fgvc7/images/'+name+'.jpg'
    img=cv2.imread(path)
    image=cv2.resize(img,(img_size,img_size),interpolation=cv2.INTER_AREA)
    test_image.append(image)
fig, ax = plt.subplots(1, 4, figsize=(15, 15))
for i in range(4):
    ax[i].set_axis_off()
    ax[i].imshow(test_image[i])    

from keras.preprocessing.image import img_to_array

x_test = np.ndarray(shape=(len(test_image), img_size, img_size, 3),dtype = np.float32)
i=0
for image in test_image:
    x_test[i]=img_to_array(image)
    x_test[i]=test_image[i]
    i=i+1
    
x_test=x_test/255
print('Test Shape: {}'.format(x_test.shape))

In [ ]:
result = model_efficientnet_b7.predict(x_test)

res = pd.DataFrame()
res['image_id'] = image_batch_test
res['healthy'] = result[:, 0]
res['multiple_diseases'] = result[:, 1]
res['rust'] = result[:, 2]
res['scab'] = result[:, 3]
res.to_csv('submission_efficientnet_b7_da_6.csv', index=False)
res.head(10)

In [ ]:
# result_2 = model_efficientnet_b7_2st.predict(x_test)

# res = pd.DataFrame()
# res['image_id'] = image_batch_test
# res['healthy'] = result_2[:, 0]
# res['multiple_diseases'] = result_2[:, 1]
# res['rust'] = result_2[:, 2]
# res['scab'] = result_2[:, 3]
# res.to_csv('submission_efficientnet_b7_2st.csv', index=False)
# res.head(10)

# ========================== Phần nháp ===============================

In [ ]:
# from matplotlib import pyplot as plt

# h = history.history

# offset = 1
# epochs = range(offset, len(h['loss']))
# plt.figure(1, figsize=(20, 6))

# plt.subplot(121)
# plt.xlabel('epochs')
# plt.ylabel('loss')
# plt.plot(epochs, h['loss'][offset:], label='train')
# plt.plot(epochs, h['val_loss'][offset:], label='val')
# plt.legend()

# plt.subplot(122)
# plt.xlabel('epochs')
# plt.ylabel('accuracy')
# plt.plot(h[f'categorical_accuracy'], label='train')
# plt.plot(h[f'val_categorical_accuracy'], label='val')
# plt.legend()
# plt.show()


# plt.figure(1, figsize=(20, 8))
# plt.subplot(121)
# plt.xlabel('epochs')
# plt.ylabel('auc')
# plt.plot(h[f'AUC'], label='train')
# plt.plot(h[f'val_AUC'], label='val')
# plt.legend()
# plt.show()

# from sklearn.metrics import roc_auc_score

# pred_test = model.predict(x_val)
# roc_sum = 0
# for i in range(4):
#     score = roc_auc_score(y_val[:, i], pred_test[:, i])
#     roc_sum += score
#     print(f'{score:.3f}')

# roc_sum /= 4
# print(f'totally:{roc_sum:.3f}')

In [ ]:
# pred = model.predict(x_val)

# pred_around = np.zeros_like(pred)
# pred_around[np.arange(len(pred)), pred.argmax(1)] = 1

In [ ]:
# print(pred_around)

In [ ]:
# print(y_val)

In [ ]:
# from sklearn.metrics import plot_confusion_matrix
# from sklearn.metrics import confusion_matrix

# # conf_mat = confusion_matrix(y_val, pred_around)
# # labels = ['Healthy', 'Multiple','Rust','Scab']
# # sns.heatmap(conf_mat, xticklabels=labels, yticklabels=labels, annot=True)
# # plot_confusion_matrix(y_val, pred, labels=['healthy','rust','scab','multiple_diseases'])
# # plot_confusion_matrix(X =y_val, y_true = pred, labels = ['healthy','rust','scab','multiple_diseases'], normalize=False)

In [ ]:
# conf_mat = confusion_matrix(y_val.argmax(axis=1), pred_around.argmax(axis=1))

In [ ]:
# conf_mat = confusion_matrix(y_val.argmax(axis=1), pred_around.argmax(axis=1))
# import seaborn as sns
# labels = ['Healthy', 'Multiple','Rust','Scab']
# sns.heatmap(conf_mat, xticklabels=labels, yticklabels=labels, annot=True)

# Phần tiến hành thực hiện trên tập dữ liệu test

In [ ]:
# dataset_test = pd.read_csv("../input/plant-pathology-2020-fgvc7/test.csv")

In [ ]:
# print(dataset_test)

In [ ]:
# image_batch_test = dataset_test['image_id']
# labels_batch_test = dataset_test[['healthy', 'multiple_diseases', 'rust', 'scab']]

In [ ]:
# test_image=[]
# for name in dataset_test['image_id']:
#     path='../input/plant-pathology-2020-fgvc7/images/'+name+'.jpg'
#     img=cv2.imread(path)
#     image=cv2.resize(img,(img_size,img_size),interpolation=cv2.INTER_AREA)
#     test_image.append(image)
# fig, ax = plt.subplots(1, 4, figsize=(15, 15))
# for i in range(4):
#     ax[i].set_axis_off()
#     ax[i].imshow(test_image[i])    

# from keras.preprocessing.image import img_to_array

# x_test = np.ndarray(shape=(len(test_image), img_size, img_size, 3),dtype = np.float32)
# i=0
# for image in test_image:
#     x_test[i]=img_to_array(image)
#     x_test[i]=test_image[i]
#     i=i+1
    
# x_test=x_test/255
# print('Test Shape: {}'.format(x_test.shape))

In [ ]:
img 

In [ ]:
# pred = model.predict(x_test)

# res = pd.DataFrame()
# res['image_id'] = image_batch_test
# res['healthy'] = pred[:, 0]
# res['multiple_diseases'] = pred[:, 1]
# res['rust'] = pred[:, 2]
# res['scab'] = pred[:, 3]
# res.to_csv('submission.csv', index=False)
# res.head(10)

In [ ]:
# # path='../input/plant-pathology-2020-fgvc7/images/Test_0.jpg'
# # img=cv2.imread(path)
# # image=cv2.resize(img,(img_size,img_size),interpolation=cv2.INTER_AREA)
# # test_image = image
# # fig, ax = plt.subplots(1, 1, figsize=(15, 15))
# # ax.imshow(test_image)    

# # from keras.preprocessing.image import img_to_array

# # x_one_test = np.ndarray(shape=(len(test_image), img_size, img_size, 3),dtype = np.float32)

# # x_one_test=img_to_array(image)
    
# # x_one_test=x_one_test/255
# # print('Test Shape: {}'.format(x_one_test.shape))


# test_image=[]
# path='../input/plant-pathology-2020-fgvc7/images/Test_0.jpg'
# img=cv2.imread(path)
# image=cv2.resize(img,(img_size,img_size),interpolation=cv2.INTER_AREA)
# test_image.append(image)
# fig, ax = plt.subplots(1, 2, figsize=(15, 15))
# for i in range(1):
#     ax[i].set_axis_off()
#     ax[i].imshow(test_image[i])    

# from keras.preprocessing.image import img_to_array

# x_test = np.ndarray(shape=(len(test_image), img_size, img_size, 3),dtype = np.float32)
# i=0
# for image in test_image:
#     x_test[i]=img_to_array(image)
#     x_test[i]=test_image[i]
#     i=i+1
    
# x_test=x_test/255
# print('Test Shape: {}'.format(x_test.shape))

# Vì sau khi xử lý với efficiennet thì đầu ra là 3x3x2560 => quá nhiều để có thể vẽ :(

In [ ]:
# img_path = '../input/plant-pathology-2020-fgvc7/images/Test_0.jpg' 
# # Define a new Model, Input= image 
# # Output= intermediate representations for all layers in the  
# # previous model after the first.
# successive_outputs = [layer.output for layer in model_efficientnet_b7.layers[2:]]
# #visualization_model = Model(img_input, successive_outputs)
# visualization_model = tf.keras.models.Model(inputs = model_efficientnet_b7.input, outputs = successive_outputs)
# #Load the input image
# img = load_img(img_path, target_size=(224, 224))
# # Convert ht image to Array of dimension (150,150,3)
# x   = img_to_array(img)                           
# x   = x.reshape((1,) + x.shape)
# # Rescale by 1/255
# x /= 255.0
# # Let's run input image through our vislauization network
# # to obtain all intermediate representations for the image.
# successive_feature_maps = visualization_model.predict(x)
# # Retrieve are the names of the layers, so can have them as part of our plot
# layer_names = [layer.name for layer in model_efficientnet_b7.layers]
# for layer_name, feature_map in zip(layer_names, successive_feature_maps):
#   print(feature_map.shape)
#   if len(feature_map.shape) == 4:
    
#     # Plot Feature maps for the conv / maxpool layers, not the fully-connected layers
   
#     n_features = feature_map.shape[-1]  # number of features in the feature map
#     size       = feature_map.shape[ 1]  # feature map shape (1, size, size, n_features)
    
#     # We will tile our images in this matrix
#     display_grid = np.zeros((size, size * n_features))
#     print(n_features)    
#     print(size)
#     print(display_grid)

    
#     # Postprocess the feature to be visually palatable
#     for i in range(n_features):
#       x  = feature_map[0, :, :, i]
#       x -= x.mean()
#       x /= x.std ()
#       x *=  64
#       x += 128
#       x  = np.clip(x, 0, 255).astype('uint8')
#       # Tile each filter into a horizontal grid
#       display_grid[:, i * size : (i + 1) * size] = x
# # Display the grid
#     scale = 20. / n_features
#     plt.figure( figsize=(scale * n_features, scale) )
#     plt.title ( layer_name )
#     plt.grid  ( False )
#     plt.imshow( display_grid, aspect='auto', cmap='viridis' )